In [ ]:
import os
import pandas as pd
from nltk import word_tokenize

import nltk
nltk.download('punkt')

from nltk.tokenize import WordPunctTokenizer

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('russian')

import string

!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
def get_data():
    corpus = []
    for filename in os.listdir():
        if filename.endswith(".txt"):
            with open (filename, 'r', encoding = 'utf-8') as f:
                text = f.read()
                corpus.append(text)
    return corpus


In [ ]:
corpus = get_data()
corpus

In [ ]:
df = pd.DataFrame(corpus, columns = ['text'])

In [ ]:
df

In [ ]:
def preprocess_data(text,stop_words = True):
    text = text.lower()
    text_tokens = WordPunctTokenizer().tokenize(text)
    spec_chars = string.punctuation + '\n\xa0«»\t—...'
    if stopwords:
        text_tokens = [token for token in text_tokens if
                   (token not in stopwords and token not in spec_chars and not any(char.isdigit() for char in token))]
    else:
         text_tokens = [token for token in text_tokens if
                   (token not in spec_chars and not any(char.isdigit() for char in token))]
    text_lemmatized = [morph.parse(token)[0].normal_form for token in text_tokens]

    return " ".join(text_lemmatized)


In [ ]:
len(stopwords)

151

In [ ]:
text_stop = preprocess_data(corpus[0],True)
for word in stopwords:
    if word in text_stop:
        print(word)

In [ ]:
preprocess_data(corpus[0],True)

'март санкт петербургский политехнический университет пётр великий национальный академия наука беларусь нанб подписать соглашение научно технический сотрудничество документ скрепить подпись ректор спбп академик ран андрей рудский первый заместитель председатель президиум нанб сергей чижик рабочий визит санкт петербургский политехнический университет пётр великий посетить первый заместитель председатель государственный комитет наука технология республика беларусь андрей косовский первый заместитель председатель президиум нанб сергей чижик гость встречать ректор спбп андрей рудский первый проректор спбп виталий сергеев директор институт машиностроение материал транспорт анатолий попович профессор высокий инженерный физический школа институт электроника телекоммуникация вадим кораблевый учёный секретарь университет дмитрий карпов сотрудничество беларусь вестись давно область аддитивный технология порошковый материал сварка один результат совместный деятельность стать опубликовать прошлый 

In [ ]:
df = df.applymap(preprocess_data)

In [ ]:
preprocess_data(df.iloc[0].text)

In [ ]:
df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 2))

In [ ]:
type(df['text'].tolist())
texts = df['text'].tolist()

In [ ]:
len(texts)

22

In [ ]:
X = vectorizer.fit_transform(texts)

In [ ]:
X.todense()

Tf-Idf

ЧАСТОТНЫЙ АНАЛИЗ

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [ ]:
tfidf = tfidf_vectorizer.fit_transform(texts) #  мне нужен не корпус, а preprocess_data?
print(tfidf) #  не понимаю, что это и что мне с этим делать

In [ ]:
tfidf.shape
tfidf.todense()[0].tolist() # зачем нам это для посика ключевых или значимых слов?

In [ ]:
tfidf_vectorizer.vocabulary_

In [ ]:
words = tfidf_vectorizer.get_feature_names()[:10]
words

In [ ]:
import numpy as np
np.argsort(tfidf_vectorizer.vocabulary_)

array([0])

In [ ]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

In [ ]:
df.text.iloc[0]

In [ ]:
feature_names = np.array(tfidf_vectorizer.get_feature_names())

for i, article in enumerate(df.text.head()):
        article_vector = tfidf[i, :]
        words = get_top_tf_idf_words(article_vector, feature_names, 10)
        print(article)
        print(words, '\n')

In [ ]:
df_2 = df.applymap

##ФИНАЛЬНЫЙ ПРОЕКТ.НАЧАЛО

##Построение модели (LDA, RAKE, YAKE)

LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, random_state=0)

In [ ]:
lda.fit(tfidf)

In [ ]:
!pip install pyldavis

In [ ]:
#Визуализируем

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.sklearn.prepare(lda,tfidf,tfidf_vectorizer)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.001541  0.003015       1        1  16.011585
3      0.004382 -0.000029       2        1  15.628145
7     -0.001781 -0.002647       3        1  13.576632
4     -0.000346  0.000076       4        1  10.252789
6     -0.000267 -0.000155       5        1   9.848378
9     -0.000178 -0.000119       6        1   9.308796
2     -0.000019  0.000267       7        1   8.367908
5     -0.000073 -0.000347       8        1   8.323174
1     -0.000083 -0.000028       9        1   4.731084
0     -0.000093 -0.000033      10        1   3.951508, topic_info=                 Term      Freq     Total Category  logprob  loglift
2575      конференция  0.000000  0.000000  Default  30.0000  30.0000
6480             соль  0.000000  0.000000  Default  29.0000  29.0000
1326           даниил  0.000000  0.000000  Default  28.0000  28.0000
5592          радость  0.000000  0.000000  Default  27.0000  27.0000
1233           гранин  0.000000  0.000000  Default  26.0000  26.0000
...               ...       ...       ...      ...      ...      ...
170         александр  0.003109  0.059465  Topic10  -8.5713   0.2801
6795            стать  0.003262  0.066947  Topic10  -8.5235   0.2094
4236            очень  0.002892  0.056549  Topic10  -8.6439   0.2578
1204  государственный  0.002837  0.056387  Topic10  -8.6630   0.2416
3875             один  0.002844  0.060493  Topic10  -8.6605   0.1738

[398 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 8, 5, 7, 10, 3, 6, 2, 1])

Не вошло в функции

In [ ]:
stopwords_2 = []
stopwords_2 = ['для', 'из']
stopwords.extend(stopwords_2)

In [ ]:
text_tokens = [token for token in text_tokens if token not in stopwords]

Ключевые слова

In [ ]:
import nltk
from collections import Counter

In [ ]:
Counter(preprocess_data(df.iloc[0].text)).most_common(15)

##RAKE

In [ ]:
# Пробуем другугю модель

!pip install nlp-rake
import nlp_rake

In [ ]:
rake = nlp_rake.Rake(stopwords = stopwords, max_words = 2, language_code="ru")
rake.apply(whole_text)



In [ ]:
whole_text = " ".join(corpus)

In [ ]:
len(corpus)

2

##Yake

In [ ]:
# Пробуем еще одну модель

!pip install yake
import yake

In [ ]:
extractor = yake.KeywordExtractor(lan= "ru", n = 2, top = 10, dedupLim=0.3)
extractor.extract_keywords(whole_text)

In [ ]:
!pip install summa
import summa

In [ ]:
keywords = summa.keywords.keywords(preprocess_data(whole_text), language = "russian", words = 10).split("\n")

In [ ]:
summa.keywords.keywords?

In [ ]:
data = {"text":[whole_text],"keywords":[keywords]}
df = pd.DataFrame(data)

In [ ]:
df

,text,keywords
0,30 марта Санкт-Петербургский политехнический у...,"[сотрудничество, научно, научный, платформа, н..."


In [ ]:
df.to_csv("Text_with_keywords",index = False)

In [ ]:
[[el1,el2,el3],[]]